### Import libraries

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import astropy.convolution as krn
import scipy.stats as stats
import sys

In [96]:
def makeHeat(screenRes, xPos, yPos):
        xMax = screenRes[0]
        yMax = screenRes[1]
        xMin = 0
        yMin = 0
        kernelPar = 50

        # Input handeling
        xlim = np.logical_and(xPos < xMax, xPos > xMin)
        ylim = np.logical_and(yPos < yMax, yPos > yMin)
        xyLim = np.logical_and(xlim, ylim)
        dataX = xPos[xyLim]
        dataX = np.floor(dataX)
        dataY = yPos[xyLim]
        dataY = np.floor(dataY)

        # initiate map and gauskernel
        gazeMap = np.zeros([int((xMax-xMin)),int((yMax-yMin))])+0.0001
        gausKernel = krn.Gaussian2DKernel(kernelPar)

        # Rescale the position vectors (if xmin or ymin != 0)
        dataX -= xMin
        dataY -= yMin

        # Now extract all the unique positions and number of samples
        xy = np.vstack((dataX, dataY)).T
        uniqueXY, idx, counts = uniqueRows(xy)
        uniqueXY = uniqueXY.astype(int)
        # populate the gazeMap
        gazeMap[uniqueXY[:,0], uniqueXY[:,1]] = counts

        # Convolve the gaze with the gauskernel
        heatMap = np.transpose(krn.convolve_fft(gazeMap,gausKernel))
        heatMap = heatMap/np.max(heatMap)

        return heatMap

def uniqueRows(x):
    y = np.ascontiguousarray(x).view(np.dtype((np.void, x.dtype.itemsize * x.shape[1])))
    _, idx, counts = np.unique(y, return_index=True, return_counts = True)
    uniques = x[idx]
    return uniques, idx, counts


def np_euclidean_distance(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.sum(np.square(y_pred - y_true), axis=-1))



### Preprocess, extract fixations and add them to the dataframe

In [97]:
sys.path.append('./FixationDetection')
from I2MC import runI2MC

# Path to data folders
path_to_folders = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer'
# path_to_folders = 'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/online/complete'

# get all folder names
folder_names = os.listdir(path_to_folders)

# read and process original datafile for each participant
for fn in folder_names:
    path = os.path.join(path_to_folders, fn, fn+'_record.csv')       
        
    df = pd.read_csv(path)        

# order frames and drop duplicate xs,ys
df = df.sort_values('frameNr')
df = df.reset_index(drop=True)
df = df.drop_duplicates(subset=['user_pred_px_x', 'user_pred_px_y'], ignore_index=True)

# get fixations for the original datafile for each participant
fixDF = runI2MC(path, plotData = False)

# add extracted fixations to the original data file (two new columns)
# for each timestamp where fixation was detected, FixXPos and FixYPos are added
idx = 0 # index of fixDF
FixXPos = np.zeros(df.shape[0])
FixYPos = np.zeros(df.shape[0])

# iterate thru the original dataframe, thru each sample
for index, row in df.iterrows():
    
    # make sure not to iterate out of range
    if idx < fixDF.shape[0]:
        
        # go to next fixation when fixation ends
        if row['sampTime'] >= np.array(fixDF.FixEnd)[idx]:
                idx += 1
        
        # make sure not to iterate out of range
        if idx < fixDF.shape[0]:
            
            # when samples are within fixation, accumulate FixXPos and FixYPos
            if row['sampTime'] >= np.array(fixDF.FixStart)[idx] and row['sampTime'] <= np.array(fixDF.FixEnd)[idx]:

                FixXPos[index] = (np.array(fixDF.XPos)[idx])
                FixYPos[index] = (np.array(fixDF.YPos)[idx])
       

 
# add fixations to original dataframe
df['FixXPos'] = np.array(FixXPos)
df['FixYPos'] = np.array(FixYPos)


# Extract only samples when the target was presented
df = df[df.event=='target_on']

# Remove samples with no fixations
df = df[(df.FixXPos!=0) & (df.FixYPos!=0)]

# Get only frames when fixation starts
df = df.drop_duplicates(subset=['FixXPos', 'FixYPos'], ignore_index=True)





Importing and processing: "C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer\2023_06_27_08_39_36\2023_06_27_08_39_36_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 3.898467779159546s to finish!


C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


### Analyze

In [104]:
# Min distance to target in pixes
pix_in_cm = df.resX[0]/df.scrW_cm[0]
minDistTarg = pix_in_cm * 3

# Get dataframe with targets on the left (leftward saccades)
df_targLeft = df[df.target=='left']

# drop fixations that are not within the target on X-axis
df_targLeft = df_targLeft[(df_targLeft.FixXPos < (df_targLeft.targetX.iloc[0] + minDistTarg)) &
                          (df_targLeft.FixXPos > (df_targLeft.targetX.iloc[0] - minDistTarg))]

# Get only the first fixations after target was presented (if more than one fixation present)
df_targLeft = df_targLeft.drop_duplicates(subset=['trialNr'],  keep='first', ignore_index=True)

# Get dataframe with targets on the right (rightward saccades)
df_targRight = df[df.target=='right']

# drop fixations that are not within the target on X-axis
df_targRight = df_targRight[(df_targRight.FixXPos > (df_targRight.targetX.iloc[0] - minDistTarg)) &
                          (df_targRight.FixXPos < (df_targRight.targetX.iloc[0] + minDistTarg))]

# Get only the first fixations after target was presented (if more than one fixation present)
df_targRight = df_targRight.drop_duplicates(subset=['trialNr'],  keep='first', ignore_index=True)

In [118]:
def plot_fixations(df, agg='median', title=''):
    
    out = df[df.condition=='arrowHeadsOutward']
    inw = df[df.condition=='arrowHeadsInward']

    if agg=='median':
        
        outX = out.FixXPos.median()
        outY = out.FixYPos.median()

        inwX = inw.FixXPos.median()
        inwY = inw.FixYPos.median()
        
    elif agg=='mean':
        
        outX = out.FixXPos.mean()
        outY = out.FixYPos.mean()

        inwX = inw.FixXPos.mean()
        inwY = inw.FixYPos.mean()
        
        

    plt.figure()
    plt.title(title);
    plt.scatter(out.FixXPos, out.FixYPos, c='blue')
    plt.scatter(inw.FixXPos, inw.FixYPos, c='orange')
    plt.scatter(out.fixationStimX, out.fixationStimY, c='red')
    plt.scatter(out.targetX, out.fixationStimY, c='green')

    plt.scatter(outX, outY, c='purple')
    plt.scatter(inwX, inwY, c='yellow')
    
    plt.xlim((0, df.resX[0]))
    plt.ylim((0, df.resY[0]))

    return outX, outY, inwX, inwY


# Plot
summary_left = plot_fixations(df_targLeft, agg='mean', title='Target Left')
summary_right = plot_fixations(df_targRight, agg='mean', title='Target Right')


(0.0, 800.0)

### Plot heatmaps per each condition
Gaze positions for all subjects are combined

In [101]:
# Import fixations into output file




### To do
1. Summarize the incomplete datasets (failed calibration, other reasons)